# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json`

In [67]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from bson import ObjectId

In [15]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [16]:
# confirm that our new database was created
mongo.list_database_names()


['admin',
 'autosaurus',
 'classDB',
 'config',
 'fruits_db',
 'local',
 'travel_db',
 'uk_food']

In [17]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [18]:
# review the collections in our new database
db.list_collection_names()


['establishments']

In [19]:
# review the collections in our new database


In [20]:
# review a document in the establishments collection
document = db["establishments"].find_one()
pprint(document)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6701dc5a6ec9ac07ba7fa330'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [21]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [22]:
# Create a dictionary for the new restaurant data
new_resto = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [49]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_resto)

In [59]:
# Check that the new restaurant was inserted
pprint(f"Inserted restaurant with ID: {result.inserted_id}")

'Inserted restaurant with ID: 670322e0b4a0ccc83df47fc8'


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [64]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

db.establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"_id": 0, "BusinessType": 1, "BusinessTypeID": 1})


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [69]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one({"_id": ObjectId("670322e0b4a0ccc83df47fc8")}, {"$set": {"BusinessTypeID": 1}})


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [70]:
# Confirm that the new restaurant was updated
db.establishments.find_one({"_id": ObjectId("670322e0b4a0ccc83df47fc8")}, {})

{'_id': ObjectId('670322e0b4a0ccc83df47fc8'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [72]:
# Find how many documents have LocalAuthorityName as "Dover"
db.establishments.count_documents({"LocalAuthorityName": "Dover"})

994

In [75]:
# Delete all documents where LocalAuthorityName is "Dover"
del_result = db.establishments.delete_many({"LocalAuthorityName": "Dover"})
pprint(f"number of documents deleted: {del_result.deleted_count}")

'number of documents deleted: 994'


In [76]:
# Check if any remaining documents include Dover
db.establishments.count_documents({"LocalAuthorityName": "Dover"})

0

In [78]:
# Check that other documents remain with 'find_one'
db.establishments.find_one()

{'_id': ObjectId('6701dc5a6ec9ac07ba7fa616'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [80]:
# Change the data type from String to Decimal for longitude and latitude
db.establishments.update_many(
    {},
    [
        {
            "$set": {
                "geocode.latitude": {"$convert": {"input": "$geocode.latitude", "to": "double"}},
                "geocode.longitude": {"$convert": {"input": "$geocode.longitude", "to": "double"}}
            }
        }
    ]
)

UpdateResult({'n': 38787, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [82]:
db.establishments.update_many(
    {},
    [
        {
            "$set": {
                "RatingValue": {
                    "$cond": {
                        "if": {"$regexMatch": {"input": "$RatingValue", "regex": "^[0-9]*$"}},  
                        "then": {"$convert": {"input": "$RatingValue", "to": "int"}},
                        "else": None  
                    }
                }
            }
        }
    ]
)

WriteError: $regexMatch needs 'input' to be of type string, full error: {'index': 0, 'code': 51104, 'errmsg': "$regexMatch needs 'input' to be of type string"}

In [ ]:
db.establishments.find_one()

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue


In [ ]:
# Check that the coordinates and rating value are now numbers
